In [1]:
#hide
#skip
%config Completer.use_jedi = False
%config IPCompleter.greedy=True
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp fastai.data.load

In [20]:
# export
# Python native modules
import os
# Third party libs
from fastcore.all import *
from fastai.torch_basics import *
# from torch.utils.data.dataloader import DataLoader as OrgDataLoader
from torchdata.datapipes.iter import HttpReader,IterDataPipe,IterableWrapper
from torch.utils.data import dataloader
from torch.utils.data.dataloader import _MultiProcessingDataLoaderIter,_SingleProcessDataLoaderIter,_DatasetKind
_loaders = (_MultiProcessingDataLoaderIter,_SingleProcessDataLoaderIter)
from torch.utils.data.dataloader_experimental import DataLoader2
# Local modules
from fastrl.fastai.loop import *

from fastai.data.load import DataLoader as FastaiDataLoader

from torchdata.datapipes.iter import HttpReader,IterDataPipe

# DataLoader helpers
> An implimentation of the torch DataLoader that instead uses Loop as a base. It
also uses experimental API from torch.data

In [5]:
bs = 4
letters = list(string.ascii_lowercase)

In [14]:
#export
def default_worker_f(worker_id):
    set_num_threads(1)
    info = get_worker_info()
    ds = info.dataset
    ds.offs = info.id
    set_seed(info.seed)
    if hasattr(ds,'wif'): ds.wif()

class MinimumDataLoader:
    def _fn_noops(self, x=None, *args, **kwargs): return x
    
    _IterableDataset_len_called,_auto_collation,collate_fn = None,False,_fn_noops
    _index_sampler,generator,prefetch_factor = Inf.count,None,2
    dataset_kind = _dataset_kind = _DatasetKind.Iterable
    drop_last = False
    
    def __init__(self, 
                 datapipe:IterDataPipe, 
                 pin_memory=False, 
                 n_workers=0, 
                 timeout=0, 
                 persistent_workers=False
                ):
        # 
        self.datapipe = datapipe 
        self.worker_init_fn=default_worker_f
        self.num_workers = n_workers
        store_attr('pin_memory,timeout,persistent_workers')
    @property
    def dataset(self): 
        "We also set self.dataset since torch~=1.11 still uses that field."
        return self.datapipe
        
    def __iter__(self): 
        
        return _loaders[self.num_workers==0](self)

    @property
    def multiprocessing_context(self): return (None,multiprocessing)[self.num_workers>0]

    @contextmanager
    def no_multiproc(self):
        old_num_workers = self.num_workers
        try:
            self.num_workers = 0
            yield self.dataset
        finally: self.num_workers = old_num_workers

_collate_types = (ndarray, Tensor, typing.Mapping, str)

In [15]:
for o in MinimumDataLoader(letters): print(o)

a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z


In [16]:
letter_pipe=IterableWrapper(letters).batch(2)
for o in MinimumDataLoader(letter_pipe): print(o)

['a', 'b']
['c', 'd']
['e', 'f']
['g', 'h']
['i', 'j']
['k', 'l']
['m', 'n']
['o', 'p']
['q', 'r']
['s', 't']
['u', 'v']
['w', 'x']
['y', 'z']


In [17]:
letter_pipe=IterableWrapper(letters).batch(16).shuffle()
for i,o in enumerate(MinimumDataLoader(letter_pipe,n_workers=3)): print(i,L(o))

0 ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']
1 ['q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
2 ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']
3 ['q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
4 ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']
5 ['q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [24]:
# There is DataLoader2 that works natively with DataPipes... So MinimumDataLoader might not be needed.
letter_pipe=IterableWrapper(letters).batch(16).shuffle().cycle()
for i,o in enumerate(DataLoader2(letter_pipe,num_workers=3)): 
    print(i,L(o))
    if i>10: break

0 [('a',), ('b',), ('c',), ('d',), ('e',), ('f',), ('g',), ('h',), ('i',), ('j',), ('k',), ('l',), ('m',), ('n',), ('o',), ('p',)]
1 [('a',), ('b',), ('c',), ('d',), ('e',), ('f',), ('g',), ('h',), ('i',), ('j',), ('k',), ('l',), ('m',), ('n',), ('o',), ('p',)]
2 [('a',), ('b',), ('c',), ('d',), ('e',), ('f',), ('g',), ('h',), ('i',), ('j',), ('k',), ('l',), ('m',), ('n',), ('o',), ('p',)]
3 [('q',), ('r',), ('s',), ('t',), ('u',), ('v',), ('w',), ('x',), ('y',), ('z',)]
4 [('q',), ('r',), ('s',), ('t',), ('u',), ('v',), ('w',), ('x',), ('y',), ('z',)]
5 [('q',), ('r',), ('s',), ('t',), ('u',), ('v',), ('w',), ('x',), ('y',), ('z',)]
6 [('q',), ('r',), ('s',), ('t',), ('u',), ('v',), ('w',), ('x',), ('y',), ('z',)]
7 [('a',), ('b',), ('c',), ('d',), ('e',), ('f',), ('g',), ('h',), ('i',), ('j',), ('k',), ('l',), ('m',), ('n',), ('o',), ('p',)]
8 [('a',), ('b',), ('c',), ('d',), ('e',), ('f',), ('g',), ('h',), ('i',), ('j',), ('k',), ('l',), ('m',), ('n',), ('o',), ('p',)]
9 [('a',), ('

In [18]:
letter_pipe=IterableWrapper(letters).batch(16).shuffle().cycle()
for i,o in enumerate(MinimumDataLoader(letter_pipe,n_workers=3)): 
    print(i,L(o))
    if i>10: break

0 ['q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
1 ['q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
2 ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']
3 ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']
4 ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']
5 ['q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
6 ['q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
7 ['q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
8 ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']
9 ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']
10 ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p']
11 ['q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [25]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script()
    notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_core.ipynb.
Converted 00_nbdev_extension.ipynb.
Converted 02_fastai.exception_test.ipynb.
Converted 02a_fastai.loop.ipynb.
Converted 02b_fastai.data.load.ipynb.
Converted 02c_fastai.data.block.ipynb.
Converted 03_callback.core.ipynb.
Converted 04_agent.ipynb.
Converted 05_data.test_async.ipynb.
Converted 05a_data.block.ipynb.
Converted 05b_data.gym.ipynb.
Converted 06a_memory.experience_replay.ipynb.
Converted 06f_memory.tensorboard.ipynb.
Converted 10a_agents.dqn.core.ipynb.
Converted 10b_agents.dqn.targets.ipynb.
Converted 10c_agents.dqn.double.ipynb.
Converted 10d_agents.dqn.dueling.ipynb.
Converted 10e_agents.dqn.categorical.ipynb.
Converted 11a_agents.policy_gradient.ppo.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
converting: /home/fastrl_user/fastrl/nbs/02b_fastai.data.load.ipynb
converting: /home/fastrl_user/fastrl/nbs/02c_fastai.data.block.ipynb
convert